In [1]:
%config Completer.use_jedi = False
import warnings

warnings.filterwarnings(action='ignore')
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


# RNN(Recurrent Neural Network) - 순환 신경망
RNN 입력값(x), 출력값(output), 상태값(state), 가중치(w), 편향값(b) 활성화 함수(tanh)로 구성된다.
<br/>
<img src="./images/RNN.png" width="1500"/>

In [2]:
inputs = np.array([[[1, 2]]])  # 입력 데이터
# print(inputs) # [[[1 2]]]
# print(inputs.shape) # (1, 1, 2)
# print(inputs.shape[0])
tf.set_random_seed(35)

tf_inputs = tf.constant(inputs, dtype=tf.float32)
sess = tf.Session()
print('입력 데이터: {}'.format(sess.run(tf_inputs)))

# BasicRNNCell() 함수의 인수로 사용할 hidden_state의 개수를 지정해서 RNN 셀을 만든다.
rnn_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=3)
print('RNN 셀의 개수: {}'.format(rnn_cell.state_size))

# RNN 셀 초기화
initial_state = tf.random_normal(shape=[inputs.shape[0], rnn_cell.state_size], mean=1.0)
# dynamic_rnn() 함수는 입력값과 RNN 셀을 받아서 실행한 결과를 출력한다.
outputs, state = tf.nn.dynamic_rnn(cell=rnn_cell, dtype=tf.float32, inputs=tf_inputs, initial_state=initial_state)
print('출력값: {}'.format(outputs))
print('상태값: {}'.format(state))

print('가중치 개수와 편향값 개수')
for v in tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES):
    print(v)
print('=' * 80)

Metal device set to: Apple M2
입력 데이터: [[[1. 2.]]]
RNN 셀의 개수: 3
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
출력값: Tensor("rnn/transpose_1:0", shape=(1, 1, 3), dtype=float32)
상태값: Tensor("rnn/while/Exit_3:0", shape=(1, 3), dtype=float32)
가중치 개수와 편향값 개수
<tf.Variable 'rnn/basic_rnn_cell/kernel:0' shape=(5, 3) dtype=float32_ref>
<tf.Variable 'rnn/basic_rnn_cell/bias:0' shape=(3,) dtype=float32_ref>


2023-03-07 23:55:37.706135: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-07 23:55:37.706472: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-07 23:55:37.709425: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-07 23:55:37.716866: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-07 23:55:37.724518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [3]:
variable_names = [v.name for v in tf.trainable_variables()]
variable_names
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output_run, state_run = sess.run([outputs, state])
    print('출력값\n', output_run, sep='')
    print('상태값\n', state_run, sep='')  # 다음 RNN 셀로 전달된다.
    print('가중치와 편향값')
    values = sess.run(variable_names)
    for k, v in zip(variable_names, values):
        print('{}\n{}'.format(k, v))

2023-03-07 23:55:37.794023: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-07 23:55:37.794041: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-07 23:55:37.797198: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-07 23:55:37.822124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


출력값
[[[-0.86154586 -0.69077307 -0.66940147]]]
상태값
[[-0.86154586 -0.69077307 -0.66940147]]
가중치와 편향값
rnn/basic_rnn_cell/kernel:0
[[-0.60581434  0.04554379 -0.05184728]
 [-0.5220578  -0.54527855  0.1978231 ]
 [-0.36936402 -0.3568974  -0.26499468]
 [ 0.6830124  -0.75265324 -0.7705655 ]
 [ 0.00107181  0.50031275 -0.33132648]]
rnn/basic_rnn_cell/bias:0
[0. 0. 0.]


2023-03-07 23:55:38.110971: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


RNN 셀의 출력값과 상태값이 동일하다.
입력값이 1 * 2 행렬이고 RNN 상태값이 1 * 3 행렬일 경우 가중치는 총 5개의 행을 가지게 된다.
입력값이 1 * 2 행렬이고 RNN 상태값이 1 * 3 행렬일 경우 편향값은 총 3개가 필요하다.